In [10]:
import requests
import base64
import json
import os
import time
from datetime import datetime
from dotenv import load_dotenv

# --- 1. Configuration ---
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please create a .env file and add RUNPOD_API_KEY=your_key")

endpoint_id = "sz4vfpd14gvjyo"
audio_file_path = "selena.wav"
workflow_file_name = "LoadAudio-VibeVoiceSS-Maya-v2.json"

# --- 2. Define API URLs ---
run_url = f"https://api.runpod.ai/v2/{endpoint_id}/run"
status_url_template = f"https://api.runpod.ai/v2/{endpoint_id}/status/"

# --- 3. Prepare the Request Payload ---

# Load and encode the audio file
print(f"Loading and encoding audio from '{audio_file_path}'...")
with open(audio_file_path, 'rb') as f:
    audio_data = f.read()
    audio_base64 = base64.b64encode(audio_data).decode('utf-8')

# Load the workflow
print(f"Loading workflow from '{workflow_file_name}'...")
with open(workflow_file_name, 'r') as f:
    workflow_json = json.load(f)

# Dynamically set the audio filename in the workflow
audio_filename = os.path.basename(audio_file_path)
workflow_json["3"]["inputs"]["audio"] = audio_filename
workflow_json["3"]["inputs"]["audioUI"] = ""

# Construct the final payload
payload = {
    "input": {
        "workflow": workflow_json,
        "images": [
            {
                "name": audio_filename,
                "image": f"data:audio/wav;base64,{audio_base64}"
            }
        ]
    }
}

# --- 4. Send the Initial API Request to Start the Job ---
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

print("Sending request to start the job...")
response = requests.post(run_url, headers=headers, json=payload)

if response.status_code == 200:
    response_data = response.json()
    job_id = response_data.get('id')
    
    if not job_id:
        print("Error: API response did not include a job ID.")
        print("Full response:", json.dumps(response_data, indent=2))
    else:
        print(f"Job successfully started with ID: {job_id}")

        # --- 5. Poll for the Job Status ---
        status_url = status_url_template + job_id
        while True:
            print("Polling for job status...")
            status_response = requests.get(status_url, headers=headers)
            status_data = status_response.json()
            job_status = status_data.get('status')

            if job_status == 'COMPLETED':
                print("Job completed successfully!")
                
                # --- 6. Process the Final Output ---
                try:
                    output = status_data.get('output', {})
                    
                    if 'audio' in output and output['audio']:
                        print("Processing audio output...")
                        for audio_item in output['audio']:
                            audio_base64 = audio_item['data']
                            original_filename = audio_item['filename']
                            
                            # Add timestamp to filename
                            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                            new_filename = f"output_{timestamp}_{original_filename}"
                            
                            # Decode and save the audio file
                            audio_bytes = base64.b64decode(audio_base64)
                            with open(new_filename, 'wb') as f:
                                f.write(audio_bytes)
                            
                            print(f"Audio successfully saved as '{new_filename}'")
                    else:
                        print("No audio output found in the response.")

                except Exception as e:
                    print(f"\nError: Could not parse output data: {e}\n")
                    print("Full response:", json.dumps(status_data, indent=2))
                
                break

            elif job_status in ['IN_QUEUE', 'IN_PROGRESS']:
                time.sleep(5) # Wait for 5 seconds before checking again
            else:
                print(f"Job execution failed, was cancelled, or timed out.")
                print("Final Status:", job_status)
                print("Full response:", json.dumps(status_data, indent=2))
                break 

else:
    print(f"Error: Initial API request failed with status code {response.status_code}")
    print("Response:", response.text)

Loading and encoding audio from 'selena.wav'...
Loading workflow from 'LoadAudio-VibeVoiceSS-Maya-v2.json'...
Sending request to start the job...
Job successfully started with ID: f1242c55-cabd-4416-948f-a79b0376ef5f-u1
Polling for job status...
Polling for job status...
Job completed successfully!
Processing audio output...
Audio successfully saved as 'output_20250921_162206_ComfyUI_00001_.flac'
